In [16]:
import langdetect
import re
import csv
import threading
import queue as queue
import multiprocessing

In [17]:
text_re = re.compile(r'"text":"(?:[^"\\]|\\.)*"')
lang = '"lang":"en"'
total_ps = 0
total_ns = 0
pos_re = re.compile(r':[-| ]?[)|D]+')
neg_re = re.compile(r':[-| ]?[(]+')

out_file = open('./data/distance_supervised_tweets', 'wb')

with open('./data/TotallyRandomTweets_1123_1130.json', 'r', encoding='utf8') as f:
    lines_processed = 0
    i=0
    for line in f:
        lines_processed += 1
        if lang in line:
#             try:
            positives = 0
            negatives = 0
            text = text_re.search(line).group()[8:-1]
            if text.startswith('RT '):
                continue
            text_unescaped_original = bytes(text, 'utf-8').decode('unicode_escape').replace(r'\/', r'/').replace('\n', ' ')
            text_unescaped = text_unescaped_original

            text_unescaped, positives = pos_re.subn('', text_unescaped)
            text_unescaped, negatives = neg_re.subn('', text_unescaped)
            if not text:
                continue
            if positives != 0 and negatives != 0:
                continue
            elif positives == 0 and negatives == 0:
                continue
            elif positives != 0:
                s = str(i)+'\tpositive\t'+text_unescaped+'\n'
#                 print(s)
                out_file.write(s.encode(encoding='utf8', errors='backslashreplace'))
                i += 1
                total_ps += 1
#                 print('positive: ', positives)
#                 print(text_unescaped_original)
#                 print(text_unescaped)
            else:
                s = str(i)+'\tnegative\t'+text_unescaped+'\n'
#                 print(s)
                out_file.write(s.encode(encoding='utf8', errors='backslashreplace'))
                i += 1
                total_ns += 1
#                 print('negative: ', negatives)
#                 print(text_unescaped_original)
#                 print(text_unescaped)
#             if i > 500:
#                 break
        if lines_processed % 50000 == 0:
            print(lines_processed, ' lines processed')
out_file.close()
print(total_ps, total_ns)

C:\Python36\lib\site-packages\ipykernel_launcher.py:22: DeprecationWarning: invalid escape sequence '\/'


50000  lines processed
100000  lines processed
150000  lines processed
200000  lines processed
250000  lines processed
300000  lines processed
350000  lines processed
400000  lines processed
450000  lines processed
500000  lines processed
550000  lines processed
600000  lines processed
650000  lines processed
700000  lines processed
750000  lines processed
800000  lines processed
850000  lines processed
900000  lines processed
950000  lines processed
1000000  lines processed
1050000  lines processed
1100000  lines processed
1150000  lines processed
1200000  lines processed
1250000  lines processed
1300000  lines processed
1350000  lines processed
1400000  lines processed
1450000  lines processed
1500000  lines processed
1550000  lines processed
1600000  lines processed
1650000  lines processed
1700000  lines processed
1750000  lines processed
1800000  lines processed
1850000  lines processed
1900000  lines processed
1950000  lines processed
2000000  lines processed
2050000  lines proce

In [18]:
total_ps = 0
total_ns = 0
pos_re = re.compile(r':[-| ]?[)|D]+')
neg_re = re.compile(r':[-| ]?[(]+')

out_file = open('./data/distance_supervised_tweets_2', 'w', encoding='utf8')

with open('./data/Sentiment Analysis Dataset.csv', 'r', encoding='utf8') as f:
    reader = csv.reader(f)
    i=0
    lines_processed = 0
    for row in reader:
        lines_processed += 1
        positives = 0
        negatives = 0
        text = row[3].lstrip().rstrip()
        if text.startswith('RT '):
            continue
        text, positives = pos_re.subn('', text)
        text, negatives = neg_re.subn('', text)
        if not text:
            continue
        if positives != 0 and negatives != 0:
            continue
        elif positives == 0 and negatives == 0:
            continue
        elif positives != 0:
            s = str(i)+'\tpositive\t'+text+'\n'
#                 print(s)
            out_file.write(s)
            i += 1
            total_ps += 1
#                 print('positive: ', positives)
#                 print(text_unescaped_original)
#                 print(text_unescaped)
        else:
            s = str(i)+'\tnegative\t'+text+'\n'
#                 print(s)
            out_file.write(s)
            i += 1
            total_ns += 1
#                 print('negative: ', negatives)
#                 print(text_unescaped_original)
#                 print(text_unescaped)
#         if i > 500:
#             break
        if lines_processed % 1000 == 0:
            print(lines_processed, ' lines processed')
out_file.close()
print(total_ps, total_ns)

346000  lines processed
669000  lines processed
1052000  lines processed
1499000  lines processed
2770 19


In [19]:
num_threads = multiprocessing.cpu_count()
total_ps = 0
total_ns = 0
total_lock = threading.Lock()
print_lock = threading.Lock()
i=0

row_queue = queue.Queue()
processed_queue = queue.Queue()

with open('./data/output_got_new.csv', 'r', encoding='utf8') as f:
    reader = csv.reader(f, delimiter=';')
    
    for row in reader:
        row_queue.put(row)

def process_csv3():
    print(threading.current_thread().name)
    global row_queue
    global processed_queue
    global total_ps
    global total_ns
    global i
    global total_lock
    global print_lock
    lines_processed = 0
    pos_re = re.compile(r':[-| ]?[)|D]+')
    neg_re = re.compile(r':[-| ]?[(]+')
    while True:
        try:
            row = row_queue.get(timeout=3)
        except:
            print(threading.current_thread().name + " left")
            return
        lines_processed += 1
        if lines_processed % 1000 == 0:
            with print_lock:
                print(threading.current_thread().name, ': ', lines_processed, ' lines processed')
        positives = False
        negatives = False
        text = row[4].lstrip().rstrip()
        if text.startswith('RT '):
            continue
        try:
            if langdetect.detect(text) != 'en':
                continue
        except:
            continue
        text, positives = pos_re.subn('', text)
        text, negatives = neg_re.subn('', text)
        if not text:
            continue
        if positives != 0 and negatives != 0:
            continue
        elif positives == 0 and negatives == 0:
            continue
        elif positives != 0:
            s = str(i)+'\tpositive\t'+text+'\n'
#                 print(s)
            processed_queue.put(s)
            with total_lock:
                i += 1
                total_ps += 1
#                 print('positive: ', positives)
#                 print(text_unescaped_original)
#                 print(text_unescaped)
        else:
            s = str(i)+'\tnegative\t'+text+'\n'
#                 print(s)
            processed_queue.put(s)
            with total_lock:
                i += 1
                total_ns += 1
#                 print('negative: ', negatives)
#                 print(text_unescaped_original)
#                 print(text_unescaped)


threads = [threading.Thread(target=process_csv3, name="Worker {}".format(i)) for i in range(num_threads)]                

for t in threads:
    t.start()
    
for t in threads:
    t.join()
    
print(total_ps, total_ns)

with open('./data/distance_supervised_tweets_3', 'w', encoding='utf8') as f:
    while not processed_queue.empty():
        s = processed_queue.get()
        f.write(s)
    


Worker 0Worker 1

Worker 2Worker 3Worker 4Worker 5Worker 6Worker 7Worker 9Worker 8Worker 10Worker 12Worker 11Worker 13Worker 14Worker 15













Worker 1 :  1000  lines processed
Worker 15 :  1000  lines processed
Worker 9 :  1000  lines processed
Worker 4 :  1000  lines processed
Worker 0 :  Worker 61000  :  lines processed 
1000  lines processed
Worker 10 :  1000Worker 8   lines processed: 
 Worker 71000  :  lines processed 
1000  lines processed
Worker 14 :  1000  lines processed
Worker 5 :  1000 Worker 3 lines processed 
:  1000  lines processed
Worker 2 :  1000  lines processed
Worker 13 :  1000  lines processed
Worker 11 :  1000Worker 12   lines processed: 
 1000  lines processed
Worker 1 :  2000  lines processed
Worker 15 :  2000  lines processed
Worker 9 :  2000  lines processed
Worker 4 :  2000  lines processed
Worker 6 :  2000  lines processedWorker 0
 :  2000  lines processed
Worker 7 :  2000Worker 10   lines processed: 
 2000  lines processed
Worker 8 :  2000  lines pr

Worker 14 :  15000  lines processed
Worker 13 :  15000  lines processed
Worker 3 :  15000  lines processed
Worker 5 :  15000  lines processedWorker 11
 :  15000  lines processed
Worker 12 :  15000  lines processed
Worker 10 :  15000  lines processed
Worker 2 :  15000  lines processed
Worker 1 :  16000  lines processed
Worker 9 :  16000  lines processed
Worker 15 :  16000  lines processed
Worker 6 :  16000  lines processed
Worker 8 :  16000  lines processed
Worker 7 :  16000Worker 0  lines processed 
:  16000  lines processed
Worker 14 :  16000  lines processed
Worker 4 :  16000  lines processed
Worker 13 :  16000  lines processedWorker 5
 :  16000  lines processed
Worker 12 : Worker 11  16000: Worker 3    lines processed16000: 
  16000 lines processed 
 lines processed
Worker 10 :  16000  lines processed
Worker 2 :  16000  lines processed
Worker 1 :  17000  lines processed
Worker 15 :  17000 Worker 6 lines processed 
:  17000  lines processed
Worker 9 :  17000  lines processed
Worker 7

Worker 1 :  30000  lines processed
Worker 9 :  30000  lines processed
Worker 6 :  30000  lines processed
Worker 8 :  30000  lines processed
Worker 15 :  30000  lines processed
Worker 7 :  30000  lines processed
Worker 0 :  30000  lines processed
Worker 4 :  30000  lines processed
Worker 14 :  30000  lines processed
Worker 13 :  30000  lines processed
Worker 3 :  30000  lines processed
Worker 5 :  30000  lines processed
Worker 12 : Worker 11  30000:    lines processed30000
  lines processed
Worker 10 :  30000  lines processed
Worker 2 :  30000  lines processed
Worker 1 :  31000  lines processed
Worker 9 :  31000  lines processed
Worker 6 :  31000  lines processed
Worker 8 :  31000  lines processed
Worker 7 :  31000  lines processed
Worker 15 :  31000  lines processed
Worker 0 :  31000  lines processed
Worker 14 :  31000  lines processed
Worker 4 :  31000Worker 13   lines processed:  
31000  lines processed
Worker 3 :  31000  lines processed
Worker 5 :  31000  lines processed
Worker 11 :

In [ ]:
print(text)

In [12]:
fu = re.compile(r':[-| ]?[(]+')
if fu.search(':)'):
    print('ds')
a, b = fu.subn('', ':)))))))jhh')
print(a, b)

:)))))))jhh 0
